In [ ]:
import csv
import pandas as pd
from itertools import chain
import sklearn_crfsuite
from sklearn.model_selection import LeaveOneOut

In [ ]:
# 定義
WINDOW_SIZE = 2
result_path = "./data/output/Chapter6"

# 記事idの読み込み
df = pd.read_csv("./data_id.csv",header=None)
data_id_list = df.T.values.tolist()[0]

corpus_file = 'corpus_5w1hs_ch6.txt' # コーパスの読み込み

# ファイル書き出し用
ex_file = "{0}/r3_extraction_result.csv".format(result_path) # 全体の結果タイプ数
f_file = "{0}/r3_failure_result.csv".format(result_path) # 失敗タイプのデータ
ex_txt = "{0}/r3_failure_result.txt".format(result_path) # 詳細結果(テキスト)

In [ ]:
# コーパス読み込み
import codecs
class CorpusReader(object):
    
    def __init__(self, path):
        with codecs.open(path, encoding='utf-8') as f:
            sent = []
            sents = []
            for line in f:
                if line == '\n':
                    sents.append(sent)
                    sent = []
                    continue
                morph_info = line.strip().split('\t')
                sent.append(morph_info) # 形態素の保存 
        train_num = int(len(sents) * 0.9) # 9割を学習に、1割をテストに
        self.__train_sents = sents[:train_num]
        self.__test_sents = sents[train_num:]
        self.__all_sents = sents
        
    def iob_sents(self, name):
        if name == 'train':
            return self.__train_sents
        elif name == 'test':
            return self.__test_sents
        elif name == 'all':
            return self.__all_sents
        else:
            return None
    

In [ ]:
# 文字種取得
def is_hiragana(ch):
    return 0x3040 <= ord(ch) <= 0x309F 
    # ひらがな：True or False

def is_katakana(ch):
    return 0x30A0 <= ord(ch) <= 0x30FF
    # カタカタ：True or False

def get_character_type(ch): # 文字種を取得する
    if ch.isspace(): # 空白の場合
        return 'ZSPACE'
    elif ch.isdigit(): # 数字の場合
        return 'ZDIGIT'
    elif ch.islower(): # 小文字の場合
        return 'ZLLET'
    elif ch.isupper(): # 大文字の場合
        return 'ZULET'
    elif is_hiragana(ch): # ひらがなの場合
        return 'HIRAG'
    elif is_katakana(ch): # カタカナの場合
        return 'KATAK'
    else: # それ以外
        return 'OTHER'

def get_character_types(string): # 文字列の文字種を変換する
    character_types = map(get_character_type, string)
    character_types_str = '-'.join(sorted(set(character_types)))

    return character_types_str

In [ ]:
# 品詞細分類の取得
def extract_pos_with_subtype(morph):
    return '-'.join(morph[1:3]) # 品詞分類(品詞大分類-小分類)

In [ ]:
# 単語を特徴量に変換する
def word2features(sent, i):
    word = sent[i][0]
    chtype = get_character_types(sent[i][0]) # 文字種取得    
    postag = extract_pos_with_subtype(sent[i]) # 品詞分類(品詞大分類-小分類)取得
    bnstlen = sent[i][3] # 固有表現
    
    # 該当単語の前後2文字の単語の特徴を用意
    features = [ 
        'bias',
        'word=' + word,
        'type=' + chtype,
        'postag=' + postag,
        'bnstlen=' + bnstlen,
    ]
    
    window_ids = []
    window_ids.extend([i for i in range(WINDOW_SIZE*(-1),0)])
    window_ids.extend([i+1 for i in range(WINDOW_SIZE)])
    
    for window_id in window_ids:
    
        if window_id < 0: # 該当単語の前をとる
            
            if i >= (window_id*-1) : 
                word = sent[i+window_id][0]
                chtype = get_character_types(sent[i+window_id][0])
                postag = extract_pos_with_subtype(sent[i+window_id])
                bnstlen = sent[i+window_id][3]
                features.extend([
                    str(window_id) + ':word=' + word,
                    str(window_id) + ':type=' + chtype,
                    str(window_id) + ':postag=' + postag,
                    str(window_id) + ':bnstlen=' + bnstlen,
                ])
            else: # それ以外は、BOS
                features.append('BOS')
                
        else: # 該当単語の後ろをとる
            if i < len(sent)-window_id:
                word = sent[i+window_id][0]
                chtype = get_character_types(sent[i+window_id][0])
                postag = extract_pos_with_subtype(sent[i+window_id])
                bnstlen = sent[i+window_id][3]
                features.extend([
                    "+" + str(window_id) + ':word=' + word,
                    "+" + str(window_id) + ':type=' + chtype,
                    "+" + str(window_id) + ':postag=' + postag,
                    "+" + str(window_id) + ':bnstlen=' + bnstlen,
                ])
            else: # それ以外は、EOS
                features.append('EOS')                        
            
    return features    

def sent2features(sent): # 情報系列から特徴を取得
    # 単語ごとに特徴変換していく
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent): # 情報系列からラベル[B、I、O]を取得
    return [morph[-1] for morph in sent]

def sent2tokens(sent): # 情報系列から単語原文を取得
    return [morph[0] for morph in sent]

In [ ]:
# ラベルごとに単語を保存する
def label_report(text,label_data):
    wh_dic = {"WHERE":[], "WHEN":[], "WHO":[], "WHAT":[], "HOW":[], "WHY":[],"SERIF":[],"O":[]}
    
    set_label = ""
    result_word = ""
    wc = -1
    wc_l = []
    
    for i,word in enumerate(text):

        ld = label_data[i]
        
        # B,Iタグをとりあえず、無視する
        if "B-" in ld:
            ld = ld.replace("B-", "")
        elif "I-" in ld:
            ld = ld.replace("I-", "")
            
        # 最後の要素のとき
        if i == (len(text)-1):
            wh_dic[set_label].append([result_word,wc_l])
            break
        
        # ラベルごとに辞書へ保存する。
        if set_label != "" and set_label != ld: 
            # 辞書に保存            
            wh_dic[set_label].append([result_word,wc_l])
            
            # 初期化
            result_word = ""
            wc_l = []

        # 同一ラベルの情報を結合
        for wi in range(len(word)):
            wc += 1
            wc_l.append(wc)
        
        # 同一ラベルの情報を結合
        set_label = ld
        result_word += word   
            
    return wh_dic
    

In [ ]:
"""
評価用
入力：正解データ辞書、予測データ辞書
出力： 5W1Hごとの失敗パターン辞書
    - 完全： 正解チャンク
    - 一部一致： 正解チャンク, 予測チャンク
    - ラベル誤り：正解チャンク, 予測チャンク, 正解ラベル, 予測ラベル
    - 抽出漏れ： 正解チャンク
    - 過度の抽出： 予測チャンク
"""
def eval_report(true_wh_dic,pred_wh_dic):
    report_data = {}
    
    # 予測ラベルのマッチングチェックリスト (マッチしたら1)
    check_pred_dic = {}
    for k,v in pred_wh_dic.items():
        if k == "O": # ラベルOのときの数え上げない
            continue
        check_pred = [0 for i in range(len(v))]
        check_pred_dic[k] = check_pred
        
    # 正解ラベルのマッチングリスト
    check_true_dic = {}
    for k,v in true_wh_dic.items():
        if k == "O": # ラベルOのときの数え上げない
            continue
        check_true = [0 for i in range(len(v))]
        check_true_dic[k] = check_true
        
        
        
    for k,v in true_wh_dic.items():
        
        if k == "O": # ラベルOのときの数え上げない
            continue
            
        report_v = {}
        result_type1 = []
        result_type2 = []
        result_type3 = []
        
        for t_num, true_text in enumerate(v):
            true_id = true_text[1] 
                
            for anoter_pk,anoter_pts in pred_wh_dic.items(): # すべての予想ラベルから検索
                 for p_num, anoter_pt in enumerate(anoter_pts):
                        
                        if anoter_pk == "O": # ラベルOのときの数え上げない
                            continue
                        
                        pred_id = anoter_pt[1]
                        match_n = list(set(true_id) & set(pred_id))  
                        
                        if k == anoter_pk and len(match_n) > 0: # 予想も正解もチャンクとラベルが同じ
                            if len(true_id) == len(pred_id):
                                check_pred_dic[k][p_num] = 1 
                                check_true_dic[k][t_num] = 1
                                result_type1.append(true_text[0]) # 完全
                            else:
                                check_pred_dic[k][p_num] = 1 
                                check_true_dic[k][t_num] = 1
                                result_type2.append([true_text[0],anoter_pt[0]]) #一部一致
                                
                        if k != anoter_pk and len(match_n) > 0: # 予想も正解もチャンクは同じだが、ラベルが違う
                            result_type3.append([true_text[0],anoter_pt[0],k,anoter_pk]) # ラベル誤り
                            check_pred_dic[anoter_pk][p_num] = 1 
                            check_true_dic[k][t_num] = 1
                        
        report_v["完全"] = result_type1
        report_v["一部一致"] = result_type2
        report_v["ラベル誤り"] = result_type3
        report_v["抽出漏れ"] = []
        report_v["過度の抽出"] = []
        report_data[k] = report_v
    
    # 抽出漏れの処理
    for k,v in check_true_dic.items():
        noexits_ture = [i for i,c in enumerate(v) if c == 0]        
        if len(noexits_ture) > 0:
            for i in noexits_ture:
                report_data[k]["抽出漏れ"].append(true_wh_dic[k][i][0]) # 抽出漏れ
                
    
    # 過度の抽出の処理
    for k,v in check_pred_dic.items():
        miss_pred = [i for i,c in enumerate(v) if c == 0]        
        if len(miss_pred) > 0:
            for i in miss_pred:
                report_data[k]["過度の抽出"].append(pred_wh_dic[k][i][0]) # 過度の抽出

    return report_data


In [ ]:
# 4パターンの数を数える
def cal_report(file_id,result,wh_label):
    wh_num = [0 for i in range(6)]
    wh_num[0] = file_id
    for wh in wh_label:
        for k,v in result[wh].items():
            if k == "完全":
                wh_num[1] += len(v)
            elif k == "一部一致":
                wh_num[2] += len(v)
            elif k == "ラベル誤り":
                wh_num[3] += len(v)
            elif k == "抽出漏れ":
                wh_num[4] += len(v)
            elif k == "過度の抽出":
                wh_num[5] += len(v)
                    
    return wh_num  

In [ ]:
"""
4タイプの結果の数を集計する
入力：ファイルid、4タイプの結果、ラベル
出力：1つのファイルにおける、ラベルごとのタイプごとの結果数
"""
def cal_report_wh(file_id,result):
    report_data= {}
    for label,wh_value in result.items():
        
        if label == "O": # ラベルOは除外する
            continue
            
        wh_num = [0 for i in range(6)]
        wh_num[0] = file_id
        for rtype,value in wh_value.items():
            if rtype == "完全":
                wh_num[1] += len(value)
            elif rtype == "一部一致":
                wh_num[2] += len(value)
            elif rtype == "ラベル誤り":
                wh_num[3] += len(value)  
            elif rtype == "抽出漏れ":
                wh_num[4] += len(value)
            elif rtype == "過度の抽出":
                wh_num[5] += len(value)
            
        report_data[label] = wh_num
        
    return report_data

In [ ]:
# テンプレートを作成する
def w2template(pred_wh_dic,text_list):
    
    template_txt = ""
    label_ids = []
    label_name = {}
    
    # 4W情報の保存
    for k,v in pred_wh_dic.items():
        if k == "O":
            continue
            
        if len(v) != 0:
            for v_i in v:
                # 要素の最初と最後
                sp_s = v_i[1][0]
                sp_e = v_i[1][-1] + 1
                
                # ラベル情報と要素
                label_name[sp_e] = k # ラベル情報保存
                label_ids.append(sp_s) # 要素の最初
                label_ids.append(sp_e) # 要素の最後
                      
    # テンプレート生成    
    data_text = text_list # 1つめのデータを利用

    for i,sp in enumerate(sorted(label_ids)):
        
        if i == 0: # 最初
            if sp != 0:
                template_txt += data_text[0:sp]
                
        elif i == (len(label_ids)-1): # 最後
            if sp < len(data_text):
                template_txt += "<{0}>".format(label_name[sp])
                template_txt += data_text[sp:len(data_text)]
                
        elif i%2 == 0:
            template_txt += data_text[start_i:sp]
        
        elif i%2 != 0:
            template_txt += "<{0}>".format(label_name[sp])
            start_i = sp  
            
    return template_txt


In [ ]:
# ラベル評価 Leave one out

ex_txt_file = open(ex_txt, 'w')  #書き込みモードでオープン
output_result = []
failure_result = []
success_result = []
wh_result = {"WHERE":[], "WHEN":[], "WHO":[], "WHAT":[], "HOW":[], "WHY":[],"SERIF":[]}

# ファイル読み込み用
c = CorpusReader(corpus_file)
all_sents = c.iob_sents('all') # データの読み込み
loo = LeaveOneOut() # LeaveOneOut呼び出し

for train_index, test_index in loo.split(all_sents):
    
    # 1. データ読み込み
    X_train = [sent2features(all_sents[i]) for i in train_index] # 学習データの特徴量
    y_train = [sent2labels(all_sents[i]) for i in train_index] # 学習データのラベル

    X_test = [sent2features(all_sents[i]) for i in test_index] # テストデータの特徴量
    y_test = [sent2labels(all_sents[i]) for i in test_index] # テストデータのラベル
            
    # 2.学習
    # CRFモデルの準備
    crf = sklearn_crfsuite.CRF(
        algorithm='lbfgs',
        c1=0.1,
        c2=0.1,
        max_iterations=100,
        all_possible_transitions=True
        )

    crf.fit(X_train, y_train) # モデルの学習
    predicted_label = crf.predict(X_test) # モデルの予測
    
    # 4.ラベル予測結果
    # 4-1. データ用意
    test_id = test_index[0]
    example_sent = all_sents[test_id] 
    file_id = data_id_list[test_id]
    text_data = sent2tokens(example_sent) # テキスト
    text_line = "".join(text_data)
    
    p_data  = predicted_label[0] # テストデータの予測結果
    c_data = y_test[0] # テストデータの正解
    
    true_wh_dic = label_report(text_data,c_data) # 5w1hラベルごとに単語を保存する
    pred_wh_dic = label_report(text_data,p_data)  # 5w1hラベルごとに単語を保存する   

    result = eval_report(true_wh_dic,pred_wh_dic) # 結果を4パターンで出力
    wh_label = ["WHERE","WHO","WHEN","WHAT"] # ラベル定義
    c_r = cal_report(file_id,result,wh_label) # 4パターンの結果をまとめる
    output_result.append(c_r)
    
    # 結果表示
    print(file_id)
    ex_txt_file.writelines("==== {0} ====\n".format(file_id))
    ex_txt_file.writelines("{0}\n".format(text_line))
    ex_txt_file.writelines("\n")
    ex_txt_file.writelines("{0}\n".format(pred_wh_dic))
    ex_txt_file.writelines("{0}\n".format(w2template(pred_wh_dic,text_line)))
    
    ex_txt_file.writelines("-----------------------------\n")
    for wh_l in wh_label:
        ex_txt_file.writelines("==== {0} ====\n".format(wh_l))
        ex_txt_file.writelines("正解：{0}\n".format([t for t in true_wh_dic[wh_l]]))
        ex_txt_file.writelines("予想：{0}\n".format([t for t in pred_wh_dic[wh_l]]))
        ex_txt_file.writelines("ラベル誤り：{0}\n".format(result[wh_l]["ラベル誤り"]))
        ex_txt_file.writelines("抽出漏れ：{0}\n".format(result[wh_l]["抽出漏れ"]))
        ex_txt_file.writelines("過度の抽出：{0}\n".format(result[wh_l]["過度の抽出"]))
   
    ex_txt_file.writelines("-----------------------------\n") 
    ex_txt_file.writelines("{0}\n".format(["記事id","完全","一部一致","ラベル誤り","抽出漏れ","過度の抽出"]))
    ex_txt_file.writelines("{0}\n".format(c_r))

    # 1記事における5w1hラベルごとの結果
    for k,v in cal_report_wh(file_id,result).items():
        wh_result[k].append(v) 
        
    # 失敗データ・成功データのリスト化
    for wh_l in wh_label:
        for rt in result[wh_l]["完全"]:
            success_result.append([file_id,wh_l,"完全",rt])
        for rt in result[wh_l]["一部一致"]:
            success_result.append([file_id,wh_l,"一部一致",rt])
        for rt in result[wh_l]["ラベル誤り"]:
            failure_result.append([file_id,wh_l,"ラベル誤り",rt])
        for rt in result[wh_l]["抽出漏れ"]:
            failure_result.append([file_id,wh_l,"抽出漏れ",rt])
        for rt in result[wh_l]["過度の抽出"]:
            failure_result.append([file_id,wh_l,"過度の抽出",rt])      

ex_txt_file.close()

In [ ]:
# ファイル書き込み

# 評価結果
with open(ex_file, 'w') as f:
    writer = csv.writer(f, lineterminator='\n') # 改行コード（\n）を指定しておく
    writer.writerow(["記事id","完全","一部一致","ラベル誤り","抽出漏れ","過度の抽出"])
    for o_data in output_result:
        writer.writerow(o_data)

# 失敗データ
with open(f_file, 'w') as f:
    writer = csv.writer(f, lineterminator='\n') # 改行コード（\n）を指定しておく
    writer.writerow(["記事id","ラベル","失敗タイプ","単語","原因"])
    for f_data in failure_result:
        writer.writerow(f_data)